In [18]:
import pandas as pd
import numpy as np


# URL del Google Sheet modificado para exportar como CSV
#csv_url = "https://docs.google.com/spreadsheets/d/1W0alu40LnXA2qdgirE-nxP8aM8JrwjMbgUkNjDZsV6o/export?format=csv&id=1W0alu40LnXA2qdgirE-nxP8aM8JrwjMbgUkNjDZsV6o&gid=205083184"
csv_url = "https://docs.google.com/spreadsheets/d/1W0alu40LnXA2qdgirE-nxP8aM8JrwjMbgUkNjDZsV6o/export?format=csv&gid=205083184"
# Definir una función personalizada para convertir texto a float, reemplazando comas por puntos y manejando '#N/A'
def to_float(val):
    try:
        return float(val.replace(',', '.'))
    except ValueError:
        if val == "#N/A":
            return np.nan
        return val

# Definir una función personalizada para convertir texto a int
def to_int(val):
    try:
        return int(val)
    except ValueError:
        return val

# Leer el Google Sheet como un DataFrame de pandas con low_memory=False y convertidores personalizados
df = pd.read_csv(csv_url, low_memory=False, converters={
    'km_corregido': to_float,
    'tiempo_corregido': to_float#,
    #'uxt_corregido': to_float
})

# Convertir la columna 'mes_anio_corregido' a tipo fecha
df['mes-anio_corregido'] = pd.to_datetime(df['mes-anio_corregido'], format='%m/%Y', errors='coerce')

# Seleccionar las columnas que contienen '_corregido' o 'corregida' en su nombre
columns_to_select = [col for col in df.columns if '_corregido' in col or 'corregida' in col]

# Crear un nuevo DataFrame con solo las columnas seleccionadas
df_selected = df[columns_to_select].copy()

# Renombrar las columnas eliminando todo después del guion bajo
df_selected.columns = [col.split('_')[0] for col in df_selected.columns]

# Forzar la columna 'motivos' a ser del tipo int
df_selected['motivos'] = pd.to_numeric(df_selected['motivos'], errors='coerce').astype('Int64')

# Mostrar las primeras filas del DataFrame filtrado
print(df_selected.dtypes)

Nombre                   object
Nivel-tension             int64
km                      float64
Cat                      object
mes-anio         datetime64[ns]
fecha                    object
tiempo                   object
Tipo-Sal                 object
Autorizada               object
uxt                     float64
motivos                   Int64
test                    float64
dtype: object


In [19]:
# Lista de columnas para las que queremos ver la suma de uxt_corregido y km_corregido
cols_to_check_sum = ['motivos']  #, 'Cat', 'Nivel-tension']

# Lista de columnas para las que queremos ver el conteo
cols_to_check_count = ['motivos']#, 'Cat']

# Crear una lista para almacenar los DataFrames pivoteados para la suma
pivot_tables_sum = []

# Crear una lista para almacenar los DataFrames pivoteados para el conteo
pivot_tables_count = []

# Generar un DataFrame pivoteado para cada columna en cols_to_check_sum
for col in cols_to_check_sum:
    pivot = pd.pivot_table(df_selected, values=['uxt'], index='mes-anio', columns=col, aggfunc='sum', fill_value=0)
    # Renombrar las columnas para incluir el nombre de la categoría
    pivot.columns = [f"Sum_uxt_{col}_{val}" for val in pivot.columns.get_level_values(1)]
    pivot_tables_sum.append(pivot)

# Generar un DataFrame pivoteado para cada columna en cols_to_check_count
for col in cols_to_check_count:
    pivot = pd.pivot_table(df_selected, values=['uxt'], index='mes-anio', columns=col, aggfunc='count', fill_value=0)
    # Renombrar las columnas para incluir el nombre de la categoría
    pivot.columns = [f"#uxt_{col}_{val}" for val in pivot.columns.get_level_values(1)]
    pivot_tables_count.append(pivot)

# Concatenar todos los DataFrames pivoteados a lo largo del eje de las columnas
final_df_sum = pd.concat(pivot_tables_sum, axis=1)
final_df_count = pd.concat(pivot_tables_count, axis=1)

# Concatenar los DataFrames de suma y conteo
final_df = pd.concat([final_df_sum, final_df_count], axis=1)

# Mostrar las primeras filas del DataFrame final
print(final_df.head())
len(final_df)


            Sum_uxt_motivos_0  Sum_uxt_motivos_1  Sum_uxt_motivos_2  \
mes-anio                                                              
2013-01-01                0.0                0.0             7474.0   
2013-02-01                0.0                0.0             3319.0   
2013-03-01                0.0                0.0             2991.0   
2013-04-01                0.0                0.0               22.0   
2013-05-01                0.0                0.0             1009.0   

            Sum_uxt_motivos_3  Sum_uxt_motivos_4  Sum_uxt_motivos_5  \
mes-anio                                                              
2013-01-01             1421.0             2681.0                0.0   
2013-02-01              531.0             1840.0                0.0   
2013-03-01             2152.0             2110.0                0.0   
2013-04-01             5978.0             3379.0                0.0   
2013-05-01             4332.0             8231.0                0.0   

    

137

### verificar que haya datos en todos los meses


In [20]:
# Crear un rango completo de meses entre enero 2013 y mayo 2024
fechas_completas = pd.date_range(start='2013-01-01', end='2024-05-31', freq='MS').strftime('%Y-%m').tolist()

# Extraer el índice del DataFrame y convertirlo en una lista de strings en el formato 'YYYY-MM'
fechas_final_df = final_df.index.strftime('%Y-%m').tolist()

# Verificar los meses que faltan
meses_faltantes = [fecha for fecha in fechas_completas if fecha not in fechas_final_df]

# Crear un DataFrame con los meses faltantes
df_meses_faltantes = pd.DataFrame(meses_faltantes, columns=['Meses Faltantes'])
df_meses_faltantes

,Meses Faltantes


### Verificar los motivos mas relevantes
Se verifica que tengan mayor cantidad de muestras

In [21]:
# Calcular la suma total por columna
column_sums = final_df.sum()

# Mostrar la suma total por columna
print(column_sums)

Sum_uxt_motivos_0          0.0
Sum_uxt_motivos_1       8495.0
Sum_uxt_motivos_2     678338.0
Sum_uxt_motivos_3    3043369.0
Sum_uxt_motivos_4     476950.0
Sum_uxt_motivos_5          0.0
Sum_uxt_motivos_6          0.0
Sum_uxt_motivos_7          0.0
#uxt_motivos_0             0.0
#uxt_motivos_1            28.0
#uxt_motivos_2           579.0
#uxt_motivos_3          2446.0
#uxt_motivos_4           623.0
#uxt_motivos_5             0.0
#uxt_motivos_6             0.0
#uxt_motivos_7             0.0
dtype: float64


### Solo consideraremos los motivos del 1 al 4

In [23]:
columnas_seleccionadas = [col for col in final_df.columns if any(f"_motivos_{i}" in col for i in range(1, 5))]

# Selecciona las columnas del DataFrame
df_seleccionado = final_df[columnas_seleccionadas]

# Muestra el DataFrame resultante
print(df_seleccionado)

df_seleccionado.to_csv("./datos_analisis.csv", index=True, sep=";")

            Sum_uxt_motivos_1  Sum_uxt_motivos_2  Sum_uxt_motivos_3  \
mes-anio                                                              
2013-01-01                0.0             7474.0             1421.0   
2013-02-01                0.0             3319.0              531.0   
2013-03-01                0.0             2991.0             2152.0   
2013-04-01                0.0               22.0             5978.0   
2013-05-01                0.0             1009.0             4332.0   
...                       ...                ...                ...   
2024-01-01                0.0                0.0             7689.0   
2024-02-01                0.0            13289.0            11269.0   
2024-03-01                0.0                0.0            37340.0   
2024-04-01                0.0                0.0            47555.0   
2024-05-01                0.0                0.0            16211.0   

            Sum_uxt_motivos_4  #uxt_motivos_1  #uxt_motivos_2  #uxt_motivos_

In [36]:
# Cargar los archivos CSV
datos_analisis = pd.read_csv('./datos_analisis.csv', sep=";")
horas_totales_mes = pd.read_csv('./horas_totales_mes.csv', sep=";")


In [37]:
# Intentar convertir las fechas sin especificar un formato primero
datos_analisis['mes-anio'] = pd.to_datetime(datos_analisis['mes-anio'], errors='coerce')
horas_totales_mes['Fecha'] = pd.to_datetime(horas_totales_mes['Fecha'], errors='coerce')

# Verificar si hubo errores en la conversión
print(datos_analisis['mes-anio'].isnull().sum(), "fechas inválidas en datos_analisis")
print(horas_totales_mes['Fecha'].isnull().sum(), "fechas inválidas en horas_totales_mes")

# Renombrar la columna 'mes-anio' en datos_analisis para que coincida con 'Fecha' en horas_totales_mes
datos_analisis.rename(columns={'mes-anio': 'Fecha'}, inplace=True)

# Unir los DataFrames por la columna 'Fecha'
df_unido = pd.merge(datos_analisis, horas_totales_mes, on='Fecha')

df_unido

0 fechas inválidas en datos_analisis
0 fechas inválidas en horas_totales_mes


,Fecha,Sum_uxt_motivos_1,Sum_uxt_motivos_2,Sum_uxt_motivos_3,Sum_uxt_motivos_4,#uxt_motivos_1,#uxt_motivos_2,#uxt_motivos_3,#uxt_motivos_4,Horas,uxt_mensual,ht
0,2013-01-01,0.0,7474.0,1421.0,2681.0,0,9,9,4,744,10902576.0,14654.0
1,2013-02-01,0.0,3319.0,531.0,1840.0,0,12,1,4,672,9847488.0,14654.0
2,2013-03-01,0.0,2991.0,2152.0,2110.0,0,10,4,5,744,10902576.0,14654.0
3,2013-04-01,0.0,22.0,5978.0,3379.0,0,1,7,5,720,10550880.0,14654.0
4,2013-05-01,0.0,1009.0,4332.0,8231.0,0,2,9,9,744,10902576.0,14654.0
...,...,...,...,...,...,...,...,...,...,...,...,...
132,2024-01-01,0.0,0.0,7689.0,8744.0,0,0,12,4,744,10902576.0,14654.0
133,2024-02-01,0.0,13289.0,11269.0,1779.0,0,8,8,4,696,10199184.0,14654.0
134,2024-03-01,0.0,0.0,37340.0,1262.0,0,0,25,7,744,10902576.0,14654.0
135,2024-04-01,0.0,0.0,47555.0,188.0,0,0,27,1,720,10550880.0,14654.0


In [38]:
df_unido.drop(columns=['Horas', 'ht', 'Sum_uxt_motivos_1' , '#uxt_motivos_1' ], inplace=True)
df_unido

,Fecha,Sum_uxt_motivos_2,Sum_uxt_motivos_3,Sum_uxt_motivos_4,#uxt_motivos_2,#uxt_motivos_3,#uxt_motivos_4,uxt_mensual
0,2013-01-01,7474.0,1421.0,2681.0,9,9,4,10902576.0
1,2013-02-01,3319.0,531.0,1840.0,12,1,4,9847488.0
2,2013-03-01,2991.0,2152.0,2110.0,10,4,5,10902576.0
3,2013-04-01,22.0,5978.0,3379.0,1,7,5,10550880.0
4,2013-05-01,1009.0,4332.0,8231.0,2,9,9,10902576.0
...,...,...,...,...,...,...,...,...
132,2024-01-01,0.0,7689.0,8744.0,0,12,4,10902576.0
133,2024-02-01,13289.0,11269.0,1779.0,8,8,4,10199184.0
134,2024-03-01,0.0,37340.0,1262.0,0,25,7,10902576.0
135,2024-04-01,0.0,47555.0,188.0,0,27,1,10550880.0


In [39]:
# Calcular disp_p
df_unido['disp_p'] = 1 - ((df_unido['Sum_uxt_motivos_2'] + df_unido['Sum_uxt_motivos_3']) / df_unido['uxt_mensual'])

# Calcular disp_f
df_unido['disp_f'] = 1 - (df_unido['Sum_uxt_motivos_4'] / df_unido['uxt_mensual'])

# Calcular disp_final
df_unido['disp_final'] = 0.8 * df_unido['disp_f'] + 0.2 * df_unido['disp_p']

df_unido

,Fecha,Sum_uxt_motivos_2,Sum_uxt_motivos_3,Sum_uxt_motivos_4,#uxt_motivos_2,#uxt_motivos_3,#uxt_motivos_4,uxt_mensual,disp_p,disp_f,disp_final
0,2013-01-01,7474.0,1421.0,2681.0,9,9,4,10902576.0,0.999184,0.999754,0.999640
1,2013-02-01,3319.0,531.0,1840.0,12,1,4,9847488.0,0.999609,0.999813,0.999772
2,2013-03-01,2991.0,2152.0,2110.0,10,4,5,10902576.0,0.999528,0.999806,0.999751
3,2013-04-01,22.0,5978.0,3379.0,1,7,5,10550880.0,0.999431,0.999680,0.999630
4,2013-05-01,1009.0,4332.0,8231.0,2,9,9,10902576.0,0.999510,0.999245,0.999298
...,...,...,...,...,...,...,...,...,...,...,...
132,2024-01-01,0.0,7689.0,8744.0,0,12,4,10902576.0,0.999295,0.999198,0.999217
133,2024-02-01,13289.0,11269.0,1779.0,8,8,4,10199184.0,0.997592,0.999826,0.999379
134,2024-03-01,0.0,37340.0,1262.0,0,25,7,10902576.0,0.996575,0.999884,0.999222
135,2024-04-01,0.0,47555.0,188.0,0,27,1,10550880.0,0.995493,0.999982,0.999084


In [58]:

import pandas as pd
import plotly.express as px

# Supongamos que df_unido ya está cargado y contiene la columna 'disp_final'

# Crear el gráfico interactivo con Plotly
fig = px.line(df_unido, x='Fecha', y='disp_final', title='Variabilidad de disp_final en el tiempo')

# Ajustar el rango del eje y para mejorar la visualización de la variabilidad
fig.update_yaxes(range=[0.994, 1.0001])

# Guardar el gráfico en un archivo HTML
fig.write_html("./disp_final_variabilidad.html")

In [59]:
df_unido.to_csv("./serie_a_analizar.csv", index=True, sep=";")